# Probability

The foundation of Bayesian statistaics is Bayes's Theorem, and the foundation of Bayes's Theorem is conditional probability.

In this chapter, we'll start with conditional probability, derive Bayes's Theorem, and demonstrate it using a real dataset. In the next chapter, we'll use Bayes's Theorem to solve problems related to conditional probability. In the chapters that follow, we'll make the transition from Bayes's Theorem to Bayesian statistcis, and I'll explain the difference.

## Probability

The Following cell downloads the data.

In [2]:
# Download the data file
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/AllenDowney/ThinkBayes2/raw/master/data/gss_bayes.csv')

In [3]:
import pandas as pd

gss = pd.read_csv('gss_bayes.csv', index_col=0)
gss.head()

,year,age,sex,polviews,partyid,indus10
caseid,,,,,,
1,1974,21.0,1,4.0,2.0,4970.0
2,1974,41.0,1,5.0,0.0,9160.0
5,1974,58.0,2,6.0,1.0,2670.0
6,1974,30.0,1,5.0,4.0,6870.0
7,1974,48.0,1,5.0,4.0,7860.0


## 은행원의 비율

In [4]:
banker = (gss['indus10'] == 6870)
banker.head()


caseid
1    False
2    False
5    False
6     True
7    False
Name: indus10, dtype: bool

In [5]:
banker.sum()

np.int64(728)

In [6]:
banker.mean()

np.float64(0.014769730168391155)

## The Probability Function

I'll put the code from the previous section in a function that takes a Boolean series and returns a probability:

In [7]:
def prob(A):
    """Computes the probability of a proposition, A.
    
    A: Boolean Series
    
    returns: probability
    """
    return A.mean()

In [8]:
prob(banker)

np.float64(0.014769730168391155)

In [9]:
# Probability of the Female
female = (gss['sex'] == 2)
female.head()

caseid
1    False
2    False
5     True
6    False
7    False
Name: sex, dtype: bool

In [10]:
prob(female)

np.float64(0.5378575776019476)

## Political Views and Parties
the other variables we'll consider are `polviews`, which describes the political views of the respondents, and `partyid`, which describes their affiliation with a political party.

The values of `polviews` are on a seven-point scale:
```
1	Extremely liberal
2	Liberal
3	Slightly liberal
4	Moderate
5	Slightly conservative
6	Conservative
7	Extremely conservative
```


I'll define `liberal` to be True for anyone whose response is "Extremely liberal", "Liberal", or "Slightly liberal".

In [11]:
liberal = (gss['polviews'] <= 3)

Here's the fraction of respondents who are liberal by this definition.

In [12]:
prob(liberal)

np.float64(0.27374721038750255)

The values of partyid are encoded like this:
```
0	Strong democrat
1	Not strong democrat
2	Independent, near democrat
3	Independent
4	Independent, near republican
5	Not strong republican
6	Strong republican
7	Other party
```

I'll define democrat to include respondents who chose "Strong democrat" or "Not strong democrat":

In [13]:
democrat = (gss['partyid'] <= 1)

And here's the fraction of respondents who are Democrats, by this definition

In [14]:
prob(democrat)

np.float64(0.3662609048488537)

## Conjunction

In [15]:
prob(banker)

np.float64(0.014769730168391155)

In [16]:
prob(democrat)

np.float64(0.3662609048488537)

respondent is a Banker and a Democrat:

In [17]:
prob(banker & democrat)

np.float64(0.004686548995739501)

conjunction to be commutatibe

In [18]:
prob(democrat & banker)

np.float64(0.004686548995739501)

## Conditional Probability

- What is the probability that a respondent is a Democrat, given that they are liberal?

In [19]:
selected = democrat[liberal]

In [20]:
prob(selected)

np.float64(0.5206403320240125)

- What is the probability that a respondent is female, given that they are a banker?

In [21]:
selected = female[banker]
prob(selected)

np.float64(0.7706043956043956)

- What is the probability that a respondent is liberal, given that they are female?

In [22]:
selected = liberal[female]
prob(selected)

np.float64(0.27581004111500884)

In [23]:
def conditional(proposition, given):
    """Conditional probability.
    
    proposition: Boolean Series
    given: Boolean Series
    
    returns: probability
    """
    return prob(proposition[given])

In [24]:
conditional(liberal, given=female)

np.float64(0.27581004111500884)

## Conditional Probability Is Not Commutative
`conditional(A, B)` is not the same as `conditional(B, A)`.

In [25]:
conditional(female, given=banker)

np.float64(0.7706043956043956)

In [26]:
conditional(banker, given=female)

np.float64(0.02116102749801969)

## Condition and Conjunction
We can combine conditional probability and conjunction.

For example, here's the probability a respondent is female, given that they are a liberal Democrat.

In [27]:
conditional(female, given = liberal & democrat)

np.float64(0.576085409252669)

About 57% of liberal Democrats are female.

And here's the probability they are a liberal female, given that they are a banker.

In [28]:
conditional(liberal & female, given=banker)

np.float64(0.17307692307692307)

## Laws of Probability
In the next few sections, we'll derive three relationships between conjunction and conditional probability:
- Theorem 1: Using a conjunction to compute a conditional probability.
- Theorem 2: Using a conditional probability to compute a conjunction.
- Theorem 3: Using `conditional(A, B) to compute `conditional(B, A).

Theorem 3 is also known as Bayes's Theorem.

I'll write these theorems using mathematical notation for probability:
- $P(A)$ is the probability of proposition.
- $P(A ~\ \mathrm{and} ~\ B)$ is the probability of the conjunction of A and B, that is, the probability that both are true.
- $P(A|B)$ is the conditional probability of $A$ given that $B$ is ture. the vertical line between $A$ and $B$ is pronounced  "given".

With that, we are ready for Theorem 1.

### Theorem 1
What fraction of bankers are female? We have already seen one way to compute the answer:
1. Use the bracket operator to select the bankers, then
2. Use `mean` to compute the fraction of bankers who are female.

In [29]:
female[banker].mean()

np.float64(0.7706043956043956)

In [30]:
conditional(female, given=banker)

np.float64(0.7706043956043956)

But thereis another way to compute this conditional probability, by computing the ratio of two probabilities:

1. The fraction of respondents who are female bankers, and
2. The fraction of respondents who are bankers.

In other words: of all the bankers, what fraction are female bankers?

In [31]:
prob(female & banker) / prob(banker)

np.float64(0.7706043956043956)

The result is the same, This example demonstrates a general rule that relates conditional probability and conjunctioon. Here's what it looks like in math notation:
$$
P(A|B) = \frac{P(A ~\ \mathrm{and} ~\ B)}{P(B)}
$$
And that's Theorem 1.

### Theorem 2
If we start with Theorem 1 and multiply both sides by $P(B)$, we get Theorem 2.

$$
P(A ~\ \mathrm{and} ~\ B) = P(B)P(A|B)
$$

This formula suggests a second way to compute a conjunction: insteand of using the & operator, we can compute the product of two probabilities.

In [32]:
prob(liberal & democrat)

np.float64(0.1425238385067965)

In [33]:
prob(democrat) * conditional(liberal, given=democrat)

np.float64(0.1425238385067965)

### Theorem 3
We have established that conjunction is commutative. In math notation, that means:

$$
P(A ~\ and ~\ B) = P(B ~\ and ~\ A)
$$

If we apply Theorem 2 to both sides, we have

$$
P(B)P(A|B) = P(A)P(B|A)
$$

Here's one way to interpret that: if you want to check $A$ and $B$, you can do it in either order:
1. You can check B first, then A conditioned B on B, or
2. You can check A first, then B conditioned on A.

If we divied through by $P(B)$, we get Theorem 3:
$$
P(A|B) = \frac{P(A)P(B|A)}{P(B)}
$$
And that is Bayes's Theorem.

In [34]:
conditional(liberal, given=banker)

np.float64(0.2239010989010989)

In [35]:
prob(liberal) * conditional(banker, given=liberal) / prob(banker)

np.float64(0.2239010989010989)

They are the same.

## The Law Of Total Probability

$$
P(A) = P(B_1andA)+P(B_2andA)
$$
In words, the total probability of A is the sum of two possibilities: either $B_1$ and $A$ are true or $B_2$ and $A$ are true. But this law applies only if $B1$ and $B2$ are:

- Mutually exclusive, which means that only one of them can be ture, and
- Collectively exhaustive, which means that one of them must be true.

In [38]:
prob(banker)

np.float64(0.014769730168391155)

In [41]:
male = (gss['sex'] == 1)

In [43]:
prob(male & banker) + prob(female & banker)

np.float64(0.014769730168391155)

Applying Theorem 2, we can also write the law of total probability like this:

$$
P(A) = P(B_1andA) + P(B_2andA) = P(B_1)P(A|B_1) + P(B_2)P(A|B_2)
$$


In [44]:
(prob(male) * conditional(banker, given=male) + 
prob(female) * conditional(banker, given=female))

np.float64(0.014769730168391153)

When there are more than two conditions, it is more concise to write the law of total probability as a summation:
$$
P(A) = \sum_{i} P(B_i)P(A|Bi)
$$

Again, this holds as long as the conditions, $B_i$ are mutually exclusive and collectively exhaustive.

In [45]:
B = gss['polviews']
B.value_counts().sort_index()

polviews
1.0     1442
2.0     5808
3.0     6243
4.0    18943
5.0     7940
6.0     7319
7.0     1595
Name: count, dtype: int64

In [48]:
i = 4
prob(B == i) * conditional(banker, given=B == i)

np.float64(0.005822682085615744)

In [47]:
sum(prob(B == i) * conditional(banker, given=B == i)
    for i in range(1, 8))

np.float64(0.014769730168391157)

## Excercises

Exercise: Use conditional to compute the following probabilities:

- What is the probability that a respondent is liberal, given that they are a Democrat?

- What is the probability that a respondent is a Democrat, given that they are liberal?

Think carefully about the order of the arguments you pass to conditional.

In [49]:
conditional(liberal, given=democrat)

np.float64(0.3891320002215698)

In [50]:
conditional(democrat, given=liberal)

np.float64(0.5206403320240125)

Exercise: There's a famous quote about young people, old people, liberals, and conservatives that goes something like:

> If you are not a liberal at 25, you have no heart. If you are not a conservative at 35, you have no brain.

Whether you agree with this proposition or not, it suggests some probabilities we can compute as an exercise. Rather than use the specific ages 25 and 35, let's define `young` and `old` as under 30 or over 65:

In [52]:
young = (gss['age'] < 30)
prob(young)

np.float64(0.19435991073240008)

In [56]:
old = (gss['age'] > 65)
old.sum()
old.mean()
prob(old)

np.float64(0.16088456076283222)

I'll define `conservative` as someone whose political views are "Conservative", "Slightly Conservative", or "Extremely Conservative".

In [57]:
conservative = (gss['polviews'] >= 5)
prob(conservative)

np.float64(0.3419354838709677)

Use `prob` and `conditional` to compute the following probabilities.

- What is the probability that a randomly chosen respondent is a young liberal?

- What is the probability that a young person is liberal?

- What fraction of respondents are old conservatives?

- What fraction of conservatives are old?

In [58]:
prob(young & liberal)

np.float64(0.06579427875836884)

In [59]:
conditional(liberal, given=young)

np.float64(0.338517745302714)

In [60]:
prob(old & conservative)

np.float64(0.062264150943396226)

In [61]:
conditional(old, given=conservative)

np.float64(0.1820932716269135)